# Import necessary libraries

In [1]:
import os  # Provides functions for interacting with the operating system (file/directory operations)
import zipfile  # Allows working with ZIP archive files (for extracting downloaded files)
import gdown  # Google Drive download utility (for downloading files from Google Drive)
import numpy as np  # Fundamental package for scientific computing (numerical operations)
import open3d as o3d  # Library for 3D data processing (mesh operations and visualization)
import trimesh  # Library for loading and processing 3D meshes (specifically for GLB/GLTF formats)
import plotly.graph_objects as go  # For creating interactive 3D visualizations (plotting)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Create directories if they don't exist inside the 'Assets' folder


In [2]:
os.makedirs('body_samples', exist_ok=True)  # Create directory for storing body meshes (won't raise error if exists)
os.makedirs('tshirt_sample', exist_ok=True)  # Create directory for storing t-shirt mesh

# Download body mesh zip file from Google Drive
gdown.download('https://drive.google.com/uc?id=14ffo8mfRYlk5XEWX_w_egO1eMHRI8pT0', 'body_samples.zip', quiet=False)
# Downloads the body mesh dataset from Google Drive (quiet=False shows progress bar)

# Download t-shirt mesh zip file from Google Drive
gdown.download('https://drive.google.com/uc?id=1Z9q5gBgBXUb2Ogsub9doisggWqVTDgEZ', 'tshirt_sample.zip', quiet=False)
# Downloads the t-shirt mesh from Google Drive

# Extract the downloaded body mesh zip
with zipfile.ZipFile('body_samples.zip', 'r') as zip_ref:  # Open zip file in read mode
    zip_ref.extractall('body_samples')  # Extract all contents to body_samples directory

# Extract the downloaded t-shirt zip
with zipfile.ZipFile('tshirt_sample.zip', 'r') as zip_ref:  # Open zip file in read mode
    zip_ref.extractall('tshirt_sample')  # Extract all contents to tshirt_sample directory

Downloading...
From: https://drive.google.com/uc?id=14ffo8mfRYlk5XEWX_w_egO1eMHRI8pT0
To: e:\Downloads\Try 3D Mesh\body_samples.zip
100%|██████████| 21.6M/21.6M [00:13<00:00, 1.59MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1Z9q5gBgBXUb2Ogsub9doisggWqVTDgEZ
From (redirected): https://drive.google.com/uc?id=1Z9q5gBgBXUb2Ogsub9doisggWqVTDgEZ&confirm=t&uuid=fd05a4fa-b068-4a2e-b675-97f94ad0f521
To: e:\Downloads\Try 3D Mesh\tshirt_sample.zip
100%|██████████| 46.6M/46.6M [00:24<00:00, 1.90MB/s]


# Search recursively for a .obj file (body mesh) in both Male and Female folders


In [3]:
body_file = next(
    (os.path.join(root, file)  # Construct full file path
     for folder in ['body_samples/Female', 'body_samples/Male']  # Search in both gender folders
     for root, _, files in os.walk(folder)  # Recursively walk through directory tree
     for file in files if file.endswith('.obj')),  # Only consider .obj files
    None  # Default value if no file found
)

# Search recursively for a .glb file (t-shirt mesh)


In [4]:
tshirt_file = next(
    (os.path.join(root, file)  # Construct full file path
     for root, _, files in os.walk('tshirt_sample')  # Recursively walk through directory
     for file in files if file.endswith('.glb')),  # Only consider .glb files
    None  # Default value if no file found
)

# Raise an error if any of the files are not found


In [5]:
if not body_file or not tshirt_file:
    raise FileNotFoundError("Body or T-shirt file not found.")  # Halt execution if files missing

# Print paths of the found files
print(f"Found body file: {body_file}")  # Display path to body mesh file
print(f"Found t-shirt file: {tshirt_file}")  # Display path to t-shirt mesh file

Found body file: body_samples/Male\Local-Generated-1.obj
Found t-shirt file: tshirt_sample\T-shirt Sample\normal_t-shirt_animated.glb


# Load the body mesh using Open3D


In [6]:
body_mesh = o3d.io.read_triangle_mesh(body_file)  # Read OBJ file into Open3D mesh object
body_mesh.compute_vertex_normals()  # Calculate vertex normals for proper lighting/shading

TriangleMesh with 10475 points and 20908 triangles.

# Load t-shirt mesh using Trimesh (GLB format)


In [7]:
tshirt_glb = trimesh.load(tshirt_file)  # Load GLB file which may contain multiple meshes

# If the GLB file contains multiple geometries, select the first one


In [8]:
tshirt_trimesh = list(tshirt_glb.geometry.values())[0] if isinstance(tshirt_glb, trimesh.Scene) else tshirt_glb
# Handles both single-mesh GLB files and scene files with multiple meshes

In [9]:
# Optionally, print a message for verification
print("Meshes loaded successfully")

Meshes loaded successfully


# Convert Trimesh t-shirt to Open3D format


In [10]:
shirt_o3d = o3d.geometry.TriangleMesh()  # Create empty Open3D mesh
shirt_o3d.vertices = o3d.utility.Vector3dVector(tshirt_trimesh.vertices)  # Copy vertices
shirt_o3d.triangles = o3d.utility.Vector3iVector(tshirt_trimesh.faces)  # Copy faces/triangles
shirt_o3d.compute_vertex_normals()  # Calculate normals for proper rendering

TriangleMesh with 36806 points and 73316 triangles.

# Compute bounding box of the body mesh


In [11]:
body_bbox = body_mesh.get_axis_aligned_bounding_box()  # Get axis-aligned bounding box
body_center = body_bbox.get_center()  # Get center point of bounding box
body_extent = body_bbox.get_extent()  # Get dimensions (width, height, depth) of bounding box

# Determine tallest dimension to align it with Y-axis (height)


In [12]:
max_dim_index = np.argmax(body_extent)  # Find which dimension (0=X,1=Y,2=Z) is largest
if max_dim_index == 0:  # If X-axis is tallest, rotate to make Y-axis tallest
    body_mesh.rotate(body_mesh.get_rotation_matrix_from_xyz((0, 0, np.pi/2)), center=body_center)
elif max_dim_index == 2:  # If Z-axis is tallest, rotate to make Y-axis tallest
    body_mesh.rotate(body_mesh.get_rotation_matrix_from_xyz((np.pi/2, 0, 0)), center=body_center)

# Flip the body if it's upside down


In [13]:
body_vertices = np.asarray(body_mesh.vertices)  # Get vertices as numpy array
y_coords = body_vertices[:, 1]  # Extract Y coordinates (height values)
top = y_coords > (body_bbox.get_max_bound()[1] - body_extent[1] * 0.2)  # Identify top 20% vertices
bottom = y_coords < (body_bbox.get_min_bound()[1] + body_extent[1] * 0.2)  # Identify bottom 20% vertices
if np.sum(bottom) > np.sum(top) * 1.5:  # If more vertices at bottom than top (upside down)
    body_mesh.rotate(body_mesh.get_rotation_matrix_from_xyz((0, 0, np.pi)), center=body_center) # Flip 180 degrees

# Normalize t-shirt orientation: rotate from Y-up to Z-up


In [14]:
shirt_bbox = shirt_o3d.get_axis_aligned_bounding_box()  # Get shirt's bounding box
shirt_center = shirt_bbox.get_center()  # Get shirt's center point
shirt_o3d.rotate(shirt_o3d.get_rotation_matrix_from_xyz((-np.pi/2, 0, 0)), center=shirt_center)  # Rotate -90° around X

TriangleMesh with 36806 points and 73316 triangles.

# Align longest shirt dimension to Y (height)


In [15]:
shirt_bbox = shirt_o3d.get_axis_aligned_bounding_box()  # Get updated bounding box
shirt_extent = shirt_bbox.get_extent()  # Get dimensions after rotation
max_dim_index = np.argmax(shirt_extent)  # Find longest dimension
if max_dim_index == 0:  # If X is longest, rotate to make Y longest
    shirt_o3d.rotate(shirt_o3d.get_rotation_matrix_from_xyz((0, 0, np.pi/2)), center=shirt_center)
elif max_dim_index == 2:  # If Z is longest, rotate to make Y longest
    shirt_o3d.rotate(shirt_o3d.get_rotation_matrix_from_xyz((np.pi/2, 0, 0)), center=shirt_center)

# Scale the shirt to match body width


In [16]:
body_bbox = body_mesh.get_axis_aligned_bounding_box()  # Get body bounding box
shirt_bbox = shirt_o3d.get_axis_aligned_bounding_box()  # Get shirt bounding box
body_width = body_bbox.get_extent()[0]  # Get body width (X dimension)
shirt_width = shirt_bbox.get_extent()[0]  # Get shirt width
scale_factor = body_width / shirt_width * 1.2  # Calculate scale factor (with 20% margin)
shirt_o3d.scale(scale_factor, center=shirt_bbox.get_center())  # Apply scaling

TriangleMesh with 36806 points and 73316 triangles.

# Translate shirt to the chest level


In [17]:
body_center = body_bbox.get_center()  # Get body center
shirt_center = shirt_o3d.get_axis_aligned_bounding_box().get_center()  # Get shirt center

In [18]:
translation = np.zeros(3)  # Initialize translation vector
translation[0] = body_center[0] - shirt_center[0]  # Align left-right (X axis)
translation[2] = (body_center[2] - shirt_center[2]) * 0.95 - 0.12  # Move slightly forward (Z axis)

In [19]:
chest_y = body_bbox.get_min_bound()[1] + body_bbox.get_extent()[1] * 0.63  # Estimate chest height (63% up body)

In [20]:
translation[1] = chest_y - shirt_center[1]  # Align shirt center to chest height (Y axis)

In [21]:
shirt_o3d.translate(translation)  # Apply translation

TriangleMesh with 36806 points and 73316 triangles.

# Flip shirt to face forward


In [22]:
shirt_o3d.rotate(shirt_o3d.get_rotation_matrix_from_xyz((np.pi, np.pi, np.pi)), center=shirt_o3d.get_center())

TriangleMesh with 36806 points and 73316 triangles.

# Visualize result using Open3D


In [23]:
# Confirm success
print("T-shirt successfully positioned on body.")  # Status message

o3d.visualization.draw_geometries([body_mesh, shirt_o3d])  # Show both meshes in Open3D viewer

T-shirt successfully positioned on body.


# Helper function to convert Open3D mesh to Plotly mesh


In [24]:
def create_mesh3d(mesh, color, opacity):
    verts = np.asarray(mesh.vertices)  # Get vertices as numpy array
    tris = np.asarray(mesh.triangles)  # Get triangles as numpy array
    return go.Mesh3d(
        x=verts[:, 0], y=verts[:, 1], z=verts[:, 2],  # X,Y,Z coordinates
        i=tris[:, 0], j=tris[:, 1], k=tris[:, 2],  # Triangle vertex indices
        color=color, opacity=opacity, flatshading=True,  # Visual properties
        lighting=dict(ambient=0.5, diffuse=0.9),  # Lighting settings
        lightposition=dict(x=100, y=200, z=0),  # Light source position
        showscale=False  # Hide color scale
    )

# Create an interactive 3D figure with Plotly


In [25]:
fig = go.Figure(data=[
    create_mesh3d(body_mesh, color='lightblue', opacity=1.0),  # Body mesh
    create_mesh3d(shirt_o3d, color='orange', opacity=0.8)  # T-shirt mesh (semi-transparent)
])

# Set layout and show visualization


In [26]:
fig.update_layout(
    scene=dict(aspectmode='data'),  # Maintain aspect ratio
    title='Virtual Try-On Visualization',  # Figure title
    margin=dict(l=0, r=0, t=30, b=0)  # Adjust margins
)
fig.show()  # Display the interactive plot

# Save result to HTML file


In [27]:
fig.write_html("virtual_tryon_result.html")  # Save as standalone HTML file
print("Saved as virtual_tryon_result.html")  # Confirmation message

Saved as virtual_tryon_result.html
